In [53]:
import os
import river
from river import ensemble, evaluate, forest, metrics, neighbors, stream, tree

In [ ]:
process = 0.0
step = 1/(len(algos)*len(all_metrics)*len(bases))
# o iter_csv lê os dados do arquivo como string, por essa
# razão é necessário converter os dados para tipos numéricos
bases = {
    'chess': {
        'converter': {
            'x': float, 'y': float, 'labels': int},
        'target': 'labels',
        'drop': None},
    'rbf': {
        'converter': {
            'x0': float, 'x1': float, 'x2': float, 'x3': float,
            'x4': float, 'x5': float, 'x6': float, 'x7': float,
            'x8': float, 'x9': float, 'labels': int},
        'target': 'labels',
        'drop': None},
    'sea': {
        'converter': {
            'x': float, 'y': float, 'labels': int},
        'target': 'labels',
        'drop': ['z']},
    'squares': {
        'converter': {
            'x': float, 'y': float, 'labels': int},
        'target': 'labels',
        'drop': None},
}

all_metrics = [
    'Accuracy',
    'Precision',
    'Recall',
    'F1',
    'GeometricMean',]

algos = {
    #'tree': tree.ExtremelyFastDecisionTreeClassifier(),
    'forest': forest.ARFClassifier(),
    # 'ensemble': ensemble.LeveragingBaggingClassifier(),
    'neighbors': neighbors.SAMKNNClassifier()
}
for a in algos:
    for b in bases:
        for m in all_metrics:

            print(process)
            process += step

            dataset = stream.iter_csv(
                f'artificial/{b}/{b}.csv',
                converters=bases[b]['converter'],
                target=bases[b]['target'],
                drop=bases[b]['drop'],)

            # o modelo SAMKNNClassifier foi adicionado manualmente ao river
            # a partir do código disponível na versão 0.10.1, por razões
            # internas do river, o modelo não foi adicionado às demais versões
            os.makedirs(f'artificial/{b}/', exist_ok=True)
            with open(f'artificial/{b}/{m}_{a}.csv', 'w') as f:
                eval = evaluate.progressive_val_score(
                    dataset,
                    model=algos[a],
                    metric=getattr(metrics, m)(),
                    delay=delay,
                    print_every=10,
                    file=f)

            try:
                reader = open(f'artificial/{b}/{m}_{a}.csv', 'r')
                dicti = {}
                Lines = reader.readlines()
                for row in Lines:
                    colunas = row.split()
                    n_answer = int(colunas[0].strip('[]').replace(',', ''))
                    metric_title = colunas[1].strip(':')
                    percentage = float(colunas[2].strip('%'))
                    if dicti.get(metric_title) is None:
                        dicti[metric_title] = {}
                    dicti[metric_title].update({n_answer: percentage})
            except OSError:
                print('Could not open/read file:', file)
                exit()

            import matplotlib.pyplot as plt

            for metric_key in dicti.keys():
                plt.figure()
                plt.title(metric_key)
                plt.xlabel('instance')
                plt.ylabel('percentage')
                plt.plot(dicti[metric_key].keys(), dicti[metric_key].values())
                plt.savefig(f'artificial/{b}/{m}_{a}.png')